In [98]:
import requests
from r2r import R2RClient

langsearch_key = "sk-47eac60b82c844fe9dbd4f7e98de4951"

client = R2RClient(base_url="http://localhost:7272", timeout=600)


def langsearch_websearch_tool(query: str, count: int = 3) -> str:
    url = "https://api.langsearch.com/v1/web-search"
    headers = {
        "Authorization": f"Bearer {langsearch_key}",
        "Content-Type": "application/json"
    }
    data = {
        "query": query,
        "freshness": "noLimit",
        "summary": True,
        "count": count
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        json_response = response.json()
        try:
            if json_response["code"] != 200 or not json_response["data"]:
                return f"Search API request failed, reason: {response.msg or 'Unknown error'}"
                
            webpages = json_response["data"]["webPages"]["value"]
            if not webpages:
                return "No relevant results found."
            
            formatted_results = ""
            for idx, page in enumerate(webpages, start=1):
                if len(page['summary']) > 1000:  # Limit content length
                    page['summary'] = page['summary'][:1000] + "..."
            
                formatted_results += (
                    f"Citation: {idx}\n"
                    f"Title: {page['name']}\n"
                    f"URL: {page['url']}\n"
                    f"Content: {page['summary']}\n"
                )
            return formatted_results.strip()
        except Exception as e:
            return f"Search API request failed, reason: Failed to parse search results {str(e)}"
    else:
        return f"Search API request failed, status code: {response.status_code}, error message: {response.text}"


gen_config = client.system.settings().results.config['completion']['generation_config']

# gen_config['tools'] = [
#     {
#         'langsearch_websearch_tool': {
#             "type": "function",
#             "function": {
#                 "name": "langsearch_websearch_tool",
#                 "description": "Search the web for information",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "query": {
#                             "type": "string",
#                             "description": "The search query"
#                         },
#                         "count": {
#                             "type": "integer",
#                             "description": "Number of results to return",
#                             "default": 3
#                         }
#                     },
#                     "required": ["query"]
#                 }
#             }
#         }
#     }
# ]
response = client.retrieval.rag(
    query="What is RAG?",
    rag_generation_config=gen_config,
).results

# Define tools in the correct format for Ollama
# tools = [{
#     "type": "function",
#     "function": {
#         "name": "langsearch_websearch_tool",
#         "description": "Search the web for information",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "query": {
#                     "type": "string",
#                     "description": "The search query"
#                 },
#                 "count": {
#                     "type": "integer",
#                     "description": "Number of results to return",
#                     "default": 3
#                 }
#             },
#             "required": ["query"]
#         }
#     }
# }]

# # Call the model with the properly formatted tools
# response = llm.chat(
#     model="llama3.1:latest",
#     options=options,
#     messages=[
#         {
#             'role': 'user',
#             'content': 'Who won the 2024 US presidential election?'
#         }
#     ],
#     tools=tools  # Pass the structured tools object
# )

In [96]:
response.metadata['choices']

[{'finish_reason': 'tool_calls',
  'index': 0,
  'message': {'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_02f5314b-28d8-4774-ac6a-150602db8930',
     'function': {'arguments': '{"query": "RAG", "count": 10}',
      'name': 'langsearch_websearch_tool'},
     'type': 'function'}],
   'structured_content': None}}]

In [92]:

if response.generated_answer == '':
    if response.metadata['choices'][0]['message']['tool_calls']:
        for tool_call in response.metadata['choices'][0]['message']['tool_calls']:
            print(tool_call['function'])
            # if tool_call["function"]["name"] == "langsearch_websearch_tool":
            #     print(tool_call["function"]["parameters"]["query"])

{'arguments': '{"query": "US elections 2024 winner", "count": "10"}', 'name': 'langsearch_websearch_tool'}


In [83]:
response.metadata['choices'][0]['message']

{'refusal': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [{'id': 'call_43aa32a6-7a7e-4b90-958a-93a1f286a352',
   'function': {'arguments': '{"query": "US elections 2024 winner", "count": "10"}',
    'name': 'langsearch_websearch_tool'},
   'type': 'function'}],
 'structured_content': None}

In [80]:
import json
func_obj = response.metadata['choices'][0]['message']['tool_calls'][0]['function']

langsearch_websearch_tool(**json.loads(func_obj["arguments"]))

"Citation: 1\nTitle: 2024 US Presidential Election Results: Live Map - ABC News\nURL: https://abcnews.go.com/Elections/2024-us-presidential-election-results-live-map\nContent: election 2024 results and live updates \n presidential results projection : trump is president - elect \n expected vote reporting : 99 % \n 226 312 \n 226 312 \n harris \n 75 , 017 , 613 \n 270 to win \n trump \n 77 , 302 , 580 \n pending results \n polls open \n dem . \n win \n gain \n rep . \n other \n counties are colored red or blue when the % expected vote reporting reaches a set threshold . this threshold varies by state and is based on patterns of past vote reporting and expectations about how the vote will report this year . \n recent pres . projections \n states by 538 race ratings \n of 51 races . . . \n democrat republican \n solid \n likely \n lean \n toss ups \n lean \n likely \n solid total \n dem . 16 4 - - - - - 20 \n rep . - - 1 5 1 3 21 31 \n other - - - - - - - 0 \n 5 are a toss up \n state dem

In [ ]:
client.retrieval.search()

In [50]:
# Handle tool calls if needed
if "tool_calls" in response['message']:
    for tool_call in response['message']['tool_calls']:
        if tool_call["function"]["name"] == "langsearch_websearch_tool":
            # Parse arguments and call the function
            args = tool_call["function"]["arguments"]
            search_results = langsearch_websearch_tool(**args)
            
            # Continue the conversation with the tool results
            final_response = llm.chat(
                model="llama3.1:latest",
                options=options,
                messages=[
                    {
                        'role': 'user',
                        'content': 'Who won the 2024 US presidential election?'
                    },
                    {
                        'role': 'assistant',
                        'content': response["message"]["content"],
                        'tool_calls': response['message']["tool_calls"]
                    },
                    {
                        'role': 'tool',
                        #'tool_call_id': tool_call["id"],
                        'name': "langsearch_websearch_tool",
                        'content': search_results
                    }
                ]
            )
            print(final_response["message"]["content"])

The winner of the 2024 US presidential election is Donald Trump. He defeated Kamala Harris in the election, winning 312 electoral votes to her 226. This was Trump's second term as President, and he became the 47th President of the United States.


In [48]:
print(search_results)

Search API request failed, reason: Failed to parse search results cannot access local variable 'page' where it is not associated with a value


In [9]:
client.documents.extract(client.documents.list().results[1].id)

R2RResults[GenericMessageResponse](results=GenericMessageResponse(message='Document entities and relationships extracted successfully.'))

In [8]:
client.documents.list_chunks(client.documents.list().results[1].id).results

[ChunkResponse(id=UUID('5b6dacdf-d51e-593c-a43f-69fab08b8b72'), document_id=UUID('ca5c75ce-0932-5735-938d-1f84a802c9d2'), owner_id=UUID('2acb499e-8428-543b-bd85-0d9098718220'), collection_ids=[UUID('122fdf6a-e116-546b-a8f6-e4cb2e2c0a09')], text='�\\n\\nMobile App*\\n\\nnotifications.\\n\\nAirport departure screens*\\n\\n.\\n\\n**Step 2: Keep Your Contact Information Updated** - Log into **“Manage My Booking”** and ensure: - Your **phone number** is correct. - Your **email address** is up-to-date.\\n\\n---\\n\\n## **Potential Issues and Resolutions for Flight Delays**\\n\\n### **1. Missed Connection Due to Delay** If the delay causes you to **miss a connecting flight** booked on the same ticket:\\n\\n**Step 1:\\n\\n** Go to the Ragas Airlines customer service desk at the airport.\\n\\n**Step 2:\\n\\n** Explain that your connecting flight was missed due to the delay.\\n\\n**Step 3:\\n\\n** The airline will rebook you on the\\n\\n**next available flight\\n\\n** at\\n\\n**no extra cost\\n\